In [ ]:
cd Prosody2Vec/

/home/dcor/niskhizov/Prosody2Vec


# Data prep

In [5]:
import torchaudio
from speechbrain.inference.speaker import EncoderClassifier
import glob
import torch
import torchaudio
from funasr import AutoModel

import numpy as np
from IPython.display import Audio



In [6]:

# model="iic/emotion2vec_base"
# model="iic/emotion2vec_base_finetuned"
# model="iic/emotion2vec_plus_seed"
# model="iic/emotion2vec_plus_base"
model_id = "iic/emotion2vec_plus_large"

sed_model = AutoModel(
    model=model_id,
    hub="ms",  # "ms" or "modelscope" for China mainland users; "hf" or "huggingface" for other overseas users
)


2025-03-01 00:01:36,141 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Detect model requirements, begin to install it: /home/dcor/niskhizov/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/requirements.txt
install model requirements successfully
ckpt: /home/dcor/niskhizov/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt


/home/dcor/niskhizov/anaconda3/lib/python3.12/site-packages/funasr/train_utils/load_pretrained_model.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  src_state = torch.l

init param, map: modality_encoders.AUDIO.extra_tokens from d2v_model.modality_encoders.AUDIO.extra_tokens in ckpt
init param, map: modality_encoders.AUDIO.alibi_scale from d2v_model.modality_encoders.AUDIO.alibi_scale in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.1.0.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.1.0.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.1.2.1.weight from d2v_model.modality_encoders.AUDIO.loc

In [7]:
import torch, torchaudio

# Load the content encoder (either hubert_soft or hubert_discrete)
hubert_discrete = torch.hub.load("bshall/hubert:main", "hubert_discrete", trust_repo=True).cuda()

# Load the content encoder (either hubert_soft or hubert_discrete)
hubert = torch.hub.load("bshall/hubert:main", "hubert_soft", trust_repo=True).cuda()

# Load the acoustic model (either hubert_soft or hubert_discrete)
acoustic = torch.hub.load("bshall/acoustic-model:main", "hubert_soft", trust_repo=True).cuda()

# Load the vocoder (either hifigan_hubert_soft or hifigan_hubert_discrete)
hifigan = torch.hub.load("bshall/hifigan:main", "hifigan_hubert_soft", trust_repo=True).cuda()


Using cache found in /home/dcor/niskhizov/cache/hub/bshall_hubert_main
Using cache found in /home/dcor/niskhizov/cache/hub/bshall_hubert_main
Using cache found in /home/dcor/niskhizov/cache/hub/bshall_acoustic-model_main
Using cache found in /home/dcor/niskhizov/cache/hub/bshall_hifigan_main
/home/dcor/niskhizov/anaconda3/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [8]:
from torch.nn import functional as F
class LogMelSpectrogram(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.melspctrogram = torchaudio.transforms.MelSpectrogram(
            sample_rate=16000,
            n_fft=1024,
            win_length=1024,
            hop_length=160,
            center=False,
            power=1.0,
            norm="slaney",
            onesided=True,
            n_mels=128,
            mel_scale="slaney",
        )

    def forward(self, wav):
        padding = (1024 - 160) // 2
        wav = F.pad(wav, (padding, padding), "reflect")
        mel = self.melspctrogram(wav)
        logmel = torch.log(torch.clamp(mel, min=1e-5))
        return logmel
    
logmel = LogMelSpectrogram().cuda()

/home/dcor/niskhizov/anaconda3/lib/python3.12/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


In [9]:

spk_ecapa_tdnn = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _load
DEBUG:speechbrain.utils.checkpoints:Registered parameter transfer hook for _load
/home/dcor/niskhizov/anaconda3/lib/python3.12/site-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load_if_possible
DEBUG:speechbrain.utils.parameter_transfer:Fetching files for pretraining (no collection directory set)
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt:

In [10]:
# data_dir = './Emotion Speech Dataset/'
# data_dir = '/home/dcor/niskhizov/Prosody2Vec/IEMOCAP_full_release/'
# data_dir = '/home/dcor/niskhizov/Prosody2Vec/Emotion Speech Dataset/0018//'
# scan recursively for all .wav files in the data_dir
data_dir = '/home/dcor/niskhizov/Prosody2Vec/Emotion Speech Dataset'

wav_files = glob.glob(data_dir + '/**/*.wav', recursive=True)

# wav_files = [x for x in wav_files if 'sentences' in x]

In [11]:

print(len(wav_files))
wav_file = wav_files[0]

wav, sr = torchaudio.load(wav_file)
wav.shape

35000


torch.Size([1, 45040])

In [12]:
Audio(wav.squeeze().numpy(), rate=sr)

In [13]:
import pickle

In [14]:
extract_dir = 'esd_3sec_embeddings'

In [15]:
mkdir $extract_dir

mkdir: cannot create directory ‘esd_3sec_embeddings’: File exists


In [16]:
def save_vecs(discrite_units, units, emo_vec, spk_vec, logmel, out_file):
    with open(out_file, 'wb') as f:
        pickle.dump({'discrite_units':discrite_units, 'units': units, 'emo_vec': emo_vec, 'spk_vec': spk_vec, 'logmel': logmel}, f)

In [17]:
import tqdm

In [18]:
# extract hubert embeddings, emotion embeddings, and speaker embeddings

for wav_path in tqdm.tqdm(wav_files):
    wav, sr = torchaudio.load(wav_path)

    # take 3 seconds of audio
    if wav.shape[1] < 3 * sr:
        continue
    wav = wav[:, :3 * sr]
    
    with torch.inference_mode():
        # Extract speech units
        discrite_units = hubert_discrete.units(wav.unsqueeze(0).cuda())
        
        units = hubert.units(wav.unsqueeze(0).cuda())

        emo_vec = sed_model.generate(wav, granularity="utterance", extract_embedding=True, disable_pbar =True)[0]['feats']

        spk_vec = spk_ecapa_tdnn.encode_batch(wav.cuda())

        melspec = logmel(wav.cuda())
    # raise
    out_file = f"{extract_dir}/{wav_path.split('/')[-1].replace('.wav', '.pkl')}"
    save_vecs(discrite_units.cpu(), units[0].cpu(), emo_vec, spk_vec[0][0].cpu(), melspec[0].cpu(), out_file)



  0%|          | 0/35000 [00:00<?, ?it/s]

100%|██████████| 35000/35000 [2:30:03<00:00,  3.89it/s]  


In [51]:
# open iemocap_embeddings_3sec/Ses01F_impro01_F002.pkl 
import pickle
with open('iemocap_embeddings_3sec/Ses01M_script01_2_F003.pkl', 'rb') as f:
    data = pickle.load(f)

In [52]:
data

{'units': tensor([[ 0.0505, -0.1387, -0.8147,  ...,  0.3883, -0.2472,  0.1950],
         [ 0.0449, -0.1292, -0.8422,  ...,  0.3391, -0.2213,  0.1285],
         [ 0.0324, -0.1755, -0.8327,  ...,  0.3289, -0.2225,  0.0441],
         ...,
         [-0.5328, -0.0546, -0.4549,  ...,  0.9544,  0.3207,  0.2705],
         [-0.5295, -0.0409, -0.4355,  ...,  0.9927,  0.2978,  0.2653],
         [-0.2779, -0.0551, -0.5089,  ...,  0.7173,  0.2878,  0.2963]]),
 'emo_vec': array([-0.0645491 , -0.18625714, -0.42167145, ..., -0.37485626,
         0.94213283,  0.5914116 ], dtype=float32),
 'spk_vec': tensor([ -8.0083,  -1.6757,  30.4501,  -1.0768, -17.8086, -39.0249,  -5.8821,
          24.2052, -57.6546, -14.0765, -37.7213, -13.6213,  55.6698,  -7.3462,
         -23.7710,  -9.1327, -29.1936,  14.9293,  26.7175,  10.2970,   0.9132,
          23.7700, -23.2739,  12.0604,  10.1882,   0.4163, -23.9248,   1.9058,
          -7.7834, -21.6377,  20.0558, -12.9531, -16.7297,   2.8233,  -9.3955,
          11.655

In [40]:
discrite_units.shape

torch.Size([150])

In [41]:
units.shape

torch.Size([1, 150, 256])